In [1]:
from IPython.display import display, HTML, Image, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import time
import random
random.seed(1)
from inputimeout import inputimeout 

import ipywidgets as widgets
from IPython.display import display

from jupyter_ui_poll import ui_events


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [3]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}


def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info


def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [5]:
def run_ANS_single(img_file,right_answer):

    empty=Image('empty.png',width=900)
    pic=Image(img_file,width=900)


    btn1 = widgets.Button(description="LEFT")
    btn2 = widgets.Button(description="RIGHT")
    
    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)

    panel = widgets.HBox([btn1, btn2])
    
    display(pic)

    
    time.sleep(0.75)
    clear_output(wait=False)

    display(empty)


    
    
    panel = widgets.HBox([btn1, btn2])
    
    
    display(panel)


    result = wait_for_event(timeout=5)

    
    if result['description'] == '':
        print("Too slow!")
        score = 0
    elif result['description'] ==  right_answer.upper():
         #print("Well done!")
        score = 1
    else:
        #print(f"Sorry the answer was {right_answer}")
        score = 0
    time.sleep(1.5)
    clear_output(wait=False)
    return score

In [7]:
def run_ANS_all():
    
    files = ["10_9yb.png", "12_14.png","12_14yb.png","12_16yb.png", "12_9.png","12_9yb.png","14_12.png","14_12yb.png","16_12.png","18_16yb.png","18_20yb.png","20_15yba.png","7_6.png","8_6by.png","8_9.png","9_10bya.png","9_12by.png"]
    answers = ["left","right","right","right","left", "left", "left", "left", "left", "left","right","left", "left", "left","right","right","right"]
    LN = [10,12,12,12,12,12,14,14,16,18,18,20,7,8,8,9,9]
    RN = [9,14,14,16,9,9,12,12,12,16,20,15,6,6,9,10,12]
    
    print("Welcome to the ANS test")
    time.sleep(2)

    print("Choose the side with more dots")
    print("Answer within 3 seconds")
    time.sleep(2)

    total = 0

    results_dict = {
        'score_or_not': [],
        'filename': [],
        'nL': [],
        'nR': [],
        'ratio': [],
        'correct': []
        }

    for i in range (0,1):

        randomlist=random.sample(range(3),3)
        #randomlist = random.sample(range(len(files)), len(files))
        #print(randomlist)
        for i in randomlist:
            
            score = run_ANS_single(files[i], answers[i]) 
            total += score

            filename = files[i]
            nR = RN[i]
            nL = LN[i]
            ratio = nL/nR
            correct = answers[i]

            results_dict['score_or_not'].append(score)
            results_dict['filename'].append(filename)
            results_dict['nR'].append(nR)
            results_dict['nL'].append(nL)
            results_dict['ratio'].append(ratio)
            results_dict['correct'].append(correct)
            
                        
    print("You scored", total)
    
    #print(results_dict)
    resultsdf = pd.DataFrame(results_dict)
    result_json = resultsdf.to_json()
    
    return result_json


You scored 2


'{"score_or_not":{"0":0,"1":1,"2":1},"filename":{"0":"10_9yb.png","1":"12_14yb.png","2":"12_14.png"},"nL":{"0":10,"1":12,"2":12},"nR":{"0":9,"1":14,"2":14},"ratio":{"0":1.1111111111,"1":0.8571428571,"2":0.8571428571},"correct":{"0":"left","1":"right","2":"right"}}'

In [49]:
def consent():
    data_consent_info = """DATA CONSENT INFORMATION:
    
    Please read:
    
    we wish to record your response data
    
    to an anonymised public data repository.
    
    Your data will be used for educational teaching purposes
    
    practising data analysis and visualisation.
    
    Please type yes in the box below if you consent to the upload."""
    
    print(data_consent_info)
    
    result = input("> ")
    
    if result == "yes":
    
        print("Thanks for your participation.")
        
        print("Please contact philip.lewis@ucl.ac.uk")
        
        print("If you have any questions or concerns")
        
        print("regarding the stored results.")
    
    else:
        
        # end code execution by raising an exception
        
        raise(Exception("User did not consent to continue test."))
    
    
    id_instructions = """
    
    Enter your anonymised ID
    
    To generate an anonymous 4-letter unique user identifier please enter:
    
    - two letters based on the initials (first and last name) of a childhood friend
    
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    
    then your unique identifer would be CBTC
    
    """
    
    print(id_instructions)
    
    user_id = input("> ")
    
    print("User entered id:", user_id)

    return

In [51]:
consent()

print("!")
time.sleep(1)

print("name:")
name=input(">")

print("age")
age=input(">")

print("gender")
gender=input(">")

result_json=run_ANS_all()

## record the basic information & send to the google form later
data_dict= {
    'name': name,
    'age': age,
    'gender': gender,
    'result_json':result_json
}

form_url = "https://docs.google.com/forms/d/e/1FAIpQLSd_XXgdBp0-KHxEo7RXx_1Z_0BCos37jj8gtTPCgrexUHue3g/viewform?usp=sf_link"
send_to_google_form(data_dict, form_url)

You scored 3
{'total_score': 3, 'filename': ['12_14.png', '12_14yb.png', '10_9yb.png'], 'nL': [12, 12, 10], 'nR': [14, 14, 9], 'ratio': [0.8571428571428571, 0.8571428571428571, 1.1111111111111112], 'correct': ['right', 'right', 'left']}


True

True